In [ ]:
# https://www.freecodecamp.org/espanol/news/8-algoritmos-de-agrupacion-en-clusteres-en-el-aprendizaje-automatico-que-todos-los-cientificos-de-datos-deben-conocer/
# https://towardsdatascience.com/math-and-intuition-behind-affinity-propagation-4ec5feae5b23

# - Habiendo tenido un consenso entre todos los puntos, cada muestra "elige" cual otra muestra la representa mejor por similitud y cercanía relativa (distancia euclídea)
# - El centroide no se deja influenciar tanto por outliers. En kmeans si, porque se quiere disminuir la varianza total del sistema. aca no.
# - No necesita un input de cuantos grupos. Entre los mismos datos llegan por consenso a la mejor partición.
# - Devuelve el ejemplo principal del grupo. No es dato menor
# - Agrupa en forma de esfera. para significado de texto, es lo ideal.
# - Todos los puntos pertenecen a un grupo
# - puedo dar preferencia a algunos textos para que sean representativos

# martz similitud (entrada): distancias eucíde y es fija
# responsabilidad (analiza cuan adecuado es el receptor de ser ejemplar. se van ajustando). calcula la similitud, castigandola con la distancia de los demás posibles ejemplares
# disponibilidad (analiza cuan adecuado es el emisor de ser ejemplar. se van ajustando). todo con distancias eucídeas/vectoreales
# asignación (salida. suma d responsabilidad y disponibilidad) Es la matriz de todas las interacciónesxinteracciónes, donde el valor máximo de cada fila (muestra), indica quien será su ejemplar (valor de la fila)

In [ ]:
#Descarga del modelo NLP BETO
!wget https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

#Instalar librerias
%pip install transformers==4.10.0 
%pip install torch==1.9.0
%pip install spacy==3.1.3
# https://huggingface.co/transformers/v3.0.2/model_doc/bert.html

In [ ]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup,BertForMaskedLM, BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap
import transformers
import torch
import re 
from transformers import BertTokenizer, BertForSequenceClassification
import torch

pd.set_option('display.max_colwidth', -1)

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
dfa = pd.read_excel('/content/drive/My Drive/Encuesta Colaborador de red - texto libre.xlsx',sheet_name='Tema varios') #Encuesta Colaborador de red - texto libre, sheets='Tema: varios')


#SELECION DEL TEMA A CLUSTERIZAR
df_c=dfa.copy()


In [ ]:
########### DESDE ACA, HASTA CLUSTERIZACIÓN, ES NUEVO

#Nueva columna con la lista de frases por los puntos
df_c['comentario_segmentado']=df_c['comentario'].apply(lambda x: re.findall('(.*?)\.',x) if re.match('(.*?)\.',x) else x )

#Genero un indice de los comentarios que cumplen con el patron definido arriba. en este caso un punto
df_c['comentario_con_punto']=df_c['comentario_segmentado'].apply(lambda x: 1 if isinstance(x, list) else 0 )

#indices de las interacciones con o sin el patron (punto)
index_sin_punto=df_c[df_c['comentario_con_punto']==0].index
index_con_punto=df_c[df_c['comentario_con_punto']==1].index


# A los que tienen el patrón, genero una columna con la cantidad de particiones que hay por interacciones
df_c.loc[index_con_punto,'len_list']=df_c['comentario_segmentado'].apply(lambda x: int(len(x)) )

#Indices que me ayudan a entender que analizar segun las particiones de cada frase
index_con_punto_len1=df_c[(df_c['comentario_con_punto']==1)&(df_c['len_list']>1)].index
index_con_punto_len2=df_c[(df_c['comentario_con_punto']==1)&(df_c['len_list']>2)].index
index_con_punto_len3=df_c[(df_c['comentario_con_punto']==1)&(df_c['len_list']>3)].index
index_con_punto_len4=df_c[(df_c['comentario_con_punto']==1)&(df_c['len_list']>4)].index


# Para los casos que cumplen con el patron, trabajo sobre la primera oración, generando luego un indice
df_c.loc[index_con_punto,'comentario_aux1']=df_c['comentario_segmentado'].apply(lambda x: x[0] if len(x)>0 else '' )
df_c.loc[index_con_punto,'index_comentario_aux1']=df_c['comentario_aux1'].apply(lambda x: 0 if str(x)=='' else 1)
index_aux1=df_c[df_c['index_comentario_aux1']==1].index

# Para los casos que cumplen con el patron, trabajo sobre la segunda oración, generando luego un indice
df_c.loc[index_con_punto_len1,'comentario_aux2']=df_c['comentario_segmentado'].apply(lambda x: x[1] if len(x)>1 else '' )
df_c.loc[index_con_punto_len1,'index_comentario_aux2']=df_c['comentario_aux2'].apply(lambda x: 0 if str(x)=='' else 1)
index_aux2=df_c[df_c['index_comentario_aux2']==1].index

# Para los casos que cumplen con el patron, trabajo sobre la tercera oración, generando luego un indice
df_c.loc[index_con_punto_len2,'comentario_aux3']=df_c['comentario_segmentado'].apply(lambda x: x[2] if len(x)>2 else '' )
df_c.loc[index_con_punto_len2,'index_comentario_aux3']=df_c['comentario_aux3'].apply(lambda x: 0 if str(x)=='' else 1)
index_aux3=df_c[df_c['index_comentario_aux3']==1].index

# Para los casos que cumplen con el patron, trabajo sobre la cuarta oración, generando luego un indice
df_c.loc[index_con_punto_len3,'comentario_aux4']=df_c['comentario_segmentado'].apply(lambda x:  x[3] if len(x)>3 else '' )
df_c.loc[index_con_punto_len3,'index_comentario_aux4']=df_c['comentario_aux4'].apply(lambda x: 0 if str(x)=='' else 1)
index_aux4=df_c[df_c['index_comentario_aux4']==1].index

# Para los casos que cumplen con el patron, trabajo sobre la quinta oración, generando luego un indice
df_c.loc[index_con_punto_len4,'comentario_aux5']=df_c['comentario_segmentado'].apply(lambda x:  x[4] if len(x)>4 else '' )
df_c.loc[index_con_punto_len4,'index_comentario_aux5']=df_c['comentario_aux5'].apply(lambda x: 0 if str(x)=='' else 1)
index_aux5=df_c[df_c['index_comentario_aux5']==1].index



df_c.reset_index(inplace=True)


#Genero un df para cada grupo, y luego los concateno
a=df_c[['index','comentario','comentario_segmentado']].iloc[index_sin_punto,:]
b=df_c[['index','comentario','comentario_aux1']].iloc[index_aux1,:]
c=df_c[['index','comentario','comentario_aux2']].iloc[index_aux2,:]
d=df_c[['index','comentario','comentario_aux3']].iloc[index_aux3,:]
f=df_c[['index','comentario','comentario_aux4']].iloc[index_aux4,:]

def col (df):
  df.columns=['index','comentario','comentario_seg']
col(a)
col(b)
col(c)
col(d)

df_last=pd.concat([a,b])
df_last=pd.concat([df_last,c])
df_last=pd.concat([df_last,d])
df_last.reset_index(inplace=True)

In [ ]:

######################## DEFINICION DE BETO/NLP  #########################
#cargo el tokenizador de beto
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

#donde voy a trabajar
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
#Vuelvo a definir la estructura del modelo donde voy a trabajar ( es la misma que en el entrenamiento)
class BERTSentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BERTSentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("pytorch/")
        self.drop = nn.Dropout(p=0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, cls_output = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
            return_dict=False
        )
        drop_output = self.drop(cls_output)
        output = self.linear(drop_output)
        return output

#Defino el proceso del modelo para analizar el texto
def classifySentiment(review_text):
    encoding_review = tokenizer.encode_plus(
        review_text,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        padding=True,
    #       pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )

    input_ids = encoding_review['input_ids'].to(device)
    attention_mask = encoding_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1) #El output es un vector de NCLASSES dimensiones (3). Aca, se va a quedar con el valor mas alto, quien será el elegido 
    #   return prediction.tolist()[0],output.tolist()[0][0],output.tolist()[0][1],output.tolist()[0][2]
    return prediction.tolist()[0]

# model = torch.load('/content/drive/My Drive/Modelos NLP - Efecty/Tema/Modelo_Tema1_2022-05-19.pkl', map_location=torch.device('cpu'))
model = torch.load('/content/drive/My Drive/Modelos NLP - Efecty/Tema/modelV_categv3_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))


In [ ]:
# aplico modelo tema:

#todo el reproceso que requiere
dic_PAPs={1:'Capacitación',2:'Convenios general',3:'Otros',
        4:'Otros',5:'Otros',6:'Solicitud Sistema/Falla dispositivos PAP',0:'PAPs',7:'Otros',8: 'Gestión efectivo PAP'}

df_last['tema']=df_last['comentario_seg'].apply(lambda x: dic_PAPs.get(classifySentiment(x)))
df_last.groupby('tema').count()

# aplico causa raiz (abajo)

In [ ]:
print(len(df_last['index'].unique()))
print(df_last.shape[0])
print(df_last.shape[0]-len(df_last['index'].unique()))
print((df_last.shape[0]/len(df_last['index'].unique()))-1)

In [ ]:
############### CLUSTERIZACIÓN (DESDE ACA, VIEJO CODIGO)

#Defino el tokenizador a utilizar. el que uso beto para entrenar. Es el diccionario de palabras a numeros.
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

#Defino el modelo BETO
model=BertForSequenceClassification.from_pretrained("pytorch/")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
list_df=[]

In [ ]:
df_c=df_last.copy()
df_c=df_c[df_c['tema']=='Otros']
df_c.drop('level_0',axis=1,inplace=True)
df_c.reset_index(inplace=True)
text=list(df_c['comentario_seg'].apply(lambda x: x.lower()))

#Extracción de matriz de significado a todas las frases CON BETO, de los textos del excel
list_df=[]
for i in text: 
  inputs = tokenizer(i, return_tensors="pt",padding='max_length', max_length=20)
  outputs = model(**inputs, labels=labels,output_hidden_states=True)
  list_df.append(outputs.hidden_states[12].detach().numpy().flatten())


#Defino un dataframe donde cada fila es una frase, representando todos los valores de la matriz de significado
df=pd.DataFrame(list_df)
df.fillna(0,inplace=True)
X=np.array(df)

# print(len(df.iloc[0].unique()))
# print(len(df.iloc[0]))

In [ ]:
from numpy import unique
from numpy import where
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.cluster import AffinityPropagation
# La Matriz de Responsabilidad: es la matriz encargada de determinar como de responsable es cada punto de nuestro conjunto de datos, es decir la importancia que tiene.
# La Matriz de Disponibilidad: La matriz encargada de determinar la cantidad de puntos que tiene un punto al rededor, es decir sus vecinos.


# definir el modelo
affinity_model = AffinityPropagation(damping=0.6,convergence_iter=40,random_state=5)
# dumping: es el factor que controla el número de clusters, va de 0.5 a 1. Cuanto mayor es el dumping, menos clusters habrá.
# converge_iter: fijamos un número de iteraciones, si el algoritmo no converge antes de este número, se para.
# preference: se basa en la matriz de responsabilidad y de disponibilidad, más preferencia, menos cantidad de clusters. Se recomienda poner una preferencia muy baja (menor de -1000) ya que si no el algoritmo inferirá una gran cantidad de clusters.
# max_iter: el número máximo de iteraciones del algoritmo
# affinity='euclidean'. Es la q va por default. sino se ingresa manualmente una matriz de similitud


# entrenar el modelo
affinity_model.fit(X)

# asignar cada punto de datos a un grupo
affinity_result = affinity_model.predict(X)

# obtener todos los grupos únicos
affinity_clusters = unique(affinity_result)

# graficar los grupos
# for affinity_cluster in affinity_clusters:
    # obtener todos los puntos de datos que caen en este grupo
    # index = where(affinity_result == affinity_cluster)
    # hacer el gráfico
    # pyplot.scatter(X[index, 0], X[index, 1])

# mostrar el gráfico
# pyplot.show()

In [ ]:
cluster_centers_indices = affinity_model.cluster_centers_indices_


In [ ]:
dict_index_centroides=dict(zip(affinity_clusters,cluster_centers_indices))
df_c['group_cluster']=affinity_result


In [ ]:
group_centroides=(pd.DataFrame(df_c['group_cluster'].value_counts().sort_values(ascending=False)[0:20]))
group_centroides.rename(columns={'group_cluster':'qty'},inplace=True)
group_centroides=group_centroides[group_centroides['qty']>sum(group_centroides['qty'])*0.07]
 #Los grupos representativos no pueden ser más de 8, y tienen que represetar al meno un 20% de las interacciones
group_centroides.reset_index(inplace=True)
group_centroides.rename(columns={'index':'group_cluster'},inplace=True)
index_comentario_centroide=[]
comentario_centroide=[]
comentario_centroide_seg=[]
for i in list(group_centroides.group_cluster):
  index_comentario_centroide.append(dict_index_centroides.get(i))
  comentario_centroide.append(df_c['comentario'][dict_index_centroides.get(i)])
  comentario_centroide_seg.append(df_c['comentario_seg'][dict_index_centroides.get(i)])

group_centroides['index_comentario_centroide']=index_comentario_centroide
group_centroides['comentario_centroide']=comentario_centroide
group_centroides['comentario_seg']=comentario_centroide_seg
group_centroides

In [ ]:
  df_c[df_c['group_cluster']==103]

In [ ]:
# Entender lo que hizo mati. mañana nos juntamos


# PARA GUARDAR
# from google.colab import drive

# drive.mount('/content/drive')
# path = '/content/drive/My Drive/df_c_cluster_sin_temas.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   df_c.to_csv(f)

In [ ]:
# OBJETIVO: GENERAR CLUSTERS QUE ME IDENTIFIQUEN LOS PRINCIPALES PROBLEMAS

# PROXIMOS PASOS / IDEAS
# OK Incluír nuevo algritmo de tema
# OK buscar patron para división de texto por qty palabras, puntos o comas, etc. (El tema se elige por las primeras 12 palabras). Para cluster, si me interesa segmentar. Hacer mas foco en la primera oración, pero si dividir los textos
# (Caro) revisar a ver si hay algo interesante en el archivo que te envio, que es la clusterización sin tema. ayuda a pensar la logica del proceso
# (Caro) Más teoria y parametrizaciones posibles del algoritmo. Ver videos, teoría de este modelo o de otros de cluster que cumplan con el mismo objetivo.
# (Caro) Pensar la logica entendiendo que estamos segmentando. cada oración puede tener mas de un tema y causa raiz asociada. Generar una segunda clusterizaicón para tener mas frases sin los primeros ejemplos seleccionados, etc.

# OBTENER DEL MODELO, EL ENTENDIMIENTO DE LOS MAYORES PROBLEMAS. Y VERLO RESUMIDO EN INTERACCIONES PUNTUALES
# (mati) Decoder. Buscar algoritmos que mejoren/limpien/resuman las oraciones ejemplares

# Despues
# - Aplicarselo a otras interacciones que no sean colaborador de red.
# - Aplicar sin definir un tema, a ver que se encuentra.
# - Selección de palabras en los ejemplos
# - extracción de frases 

In [ ]:

#Decoder

import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'
tokenizer = BertTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

In [ ]:

def generate_summary(text):
   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=20, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids,length_penalty=0.1, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

decoder=[]
for text in group_centroides['comentario_centroide']:
  decoder.append(generate_summary(text))

decoder